In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
true_dataset=pd.read_csv('/content/drive/MyDrive/FakeNewsData/True.csv')
true_dataset.head()


In [ ]:
fake_dataset=pd.read_csv('/content/drive/MyDrive/FakeNewsData/Fake.csv')
fake_dataset.head()

In [ ]:
true_dataset.isnull().sum()

In [ ]:
fake_dataset.isnull().sum()

In [ ]:
true_dataset.duplicated().sum()

In [ ]:
true_dataset.drop_duplicates(inplace=True)

In [ ]:
true_dataset.duplicated().sum()

In [ ]:
fake_dataset.duplicated().sum()

In [ ]:
fake_dataset.drop_duplicates(inplace=True)

In [ ]:
fake_dataset.duplicated().sum()

In [ ]:
true_dataset['label']=1
fake_dataset['label']=0

In [ ]:
true_dataset.head()

In [ ]:
dataset= pd.concat([true_dataset, fake_dataset], ignore_index=True)
dataset.head(20)

In [ ]:
dataset.tail(20)

In [ ]:
dataset.shape

In [ ]:
def extract_text_after(text, delimiter):
    """
    Extracts text after a specified delimiter.

    Args:
    - text (str): The input text.
    - delimiter (str): The delimiter after which text should be extracted.

    Returns:
    - str: The text extracted after the delimiter. If delimiter is not found, returns an empty string.
    """
    index = text.find(delimiter)
    if index != -1:
        return text[index + len(delimiter):]
    else:
        return ""

# # Example usage:
# input_text = "This is a sample text - and this is the text to extract."
delimiter = "-"
# extracted_text = extract_text_after(input_text, delimiter)
# print(extracted_text)


In [ ]:
dataset['text'] = dataset['text'].apply(lambda x: extract_text_after(x, delimiter))
dataset.head()



In [ ]:
dataset.head(20)

In [ ]:
dataset['news']=dataset['title']+dataset['text']
dataset.head()

In [ ]:
X=dataset['news']
y=dataset['label']

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer



In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download necessary resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')

# Data preprocessing
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    words = [word for word in words if word.isalnum()]
    words = [word for word in words if word not in stop_words]
    # Stemming (optional)
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)


In [ ]:
X=X.apply(preprocess_text)
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

In [ ]:
# Training the classifier
text_clf.fit(X_train, y_train)

In [ ]:
# Predictions
predictions = text_clf.predict(X_test)

In [ ]:
# Evaluating the model
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, predictions))

In [ ]:
X_train

In [ ]:
y_train

In [ ]:

new_text = dataset['news'][38158]

preprocessed_text = preprocess_text(new_text)

prediction = text_clf.predict([preprocessed_text])

# Output the prediction
print("Prediction:", prediction)

In [ ]:
dataset['news'][38158]

In [ ]:
dataset['label'][5219]

LSTM

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [ ]:
# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Convert text to sequences
sequences = tokenizer.texts_to_sequences(X)

In [ ]:
# Padding sequences
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')


In [ ]:
# Splitting data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences,y, test_size=0.2, random_state=42)


In [ ]:
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Training the model
model.fit(X_train, y_train, epochs=25, batch_size=64)

Epoch 1/25
559/559 [==============================] - 124s 217ms/step - loss: 0.1379 - accuracy: 0.9480
Epoch 2/25
559/559 [==============================] - 109s 195ms/step - loss: 0.0409 - accuracy: 0.9871
Epoch 3/25
559/559 [==============================] - 105s 187ms/step - loss: 0.0189 - accuracy: 0.9945
Epoch 4/25
559/559 [==============================] - 101s 180ms/step - loss: 0.0132 - accuracy: 0.9961
Epoch 5/25
559/559 [==============================] - 97s 174ms/step - loss: 0.0106 - accuracy: 0.9973
Epoch 6/25
559/559 [==============================] - 97s 174ms/step - loss: 0.0078 - accuracy: 0.9976
Epoch 7/25
559/559 [==============================] - 96s 171ms/step - loss: 0.0092 - accuracy: 0.9975
Epoch 8/25
559/559 [==============================] - 96s 172ms/step - loss: 0.0065 - accuracy: 0.9983
Epoch 9/25
559/559 [==============================] - 95s 170ms/step - loss: 0.0016 - accuracy: 0.9996
Epoch 10/25
559/559 [==============================] - 96s 172ms/step

In [ ]:
# Evaluating the model
loss_LSTM, accuracy_LSTM = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:

new_text = dataset['news'][5219]

preprocessed_text = preprocess_text(new_text)

new_seq = tokenizer.texts_to_sequences([preprocessed_text])

new_padded_seq = pad_sequences(new_seq, maxlen=max_length, padding='pre')

prediction = model.predict(new_padded_seq)

# If the prediction is very close to 0, mark it as 0
if prediction < 1e-5:
    prediction = 0.0
# If the prediction is very close to 1, mark it as 1
elif prediction > 1 - 1e-5:
    prediction = 1.0

print("Prediction:", prediction)

Bi-LSTM

In [ ]:
from tensorflow.keras.layers import Bidirectional

In [ ]:
# Bi-LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Training the model
model.fit(X_train, y_train, epochs=25, batch_size=64)


In [ ]:
# Evaluating the model
loss_BiLSTM, accuracy_BiLSTM = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

data = {
    'Model': ['Naive Bayes', 'LSTM', 'Bi-LSTM'],
    'Accuracy': [accuracy, accuracy_LSTM, accuracy_BilSTM]
}
df = pd.DataFrame(data)

# Plotting
sns.barplot(x='Model', y='Accuracy', data=df)
plt.ylim(0, 1)  # Set y-axis limit between 0 and 1 for accuracy
plt.title('Accuracy Comparison of Different Models')
plt.show()